## Virtual Environment for Individual-Based Modeling

This virtual environment is based on an Agent-Based Modeling system using some assumptions that are derived from previousobservations of waterbirds’ behaviors within some habitats in the tropics.

Created on March 31, 2019 by Ralph Florent <r.florent@jacobs-university.de>

Last updated on January 6, 2020

In [ ]:
# Virtual Environment for Individual-Based Modeling (IBM)
#
# Created on March 31, 2019
# Last updated on Jan 6, 2020
#
# Authors:
#   Ralph Florent <r.florent@jacobs-university.de>
#   Davi Tavares <davi.tavares@leibniz-zmt.de>
#   Agostino Merico <a.merico@jacobs-university.de>
#
# Main entry point for the application

# ==============================================================================
# START: Preamble
# ==============================================================================
%reset

# -*- coding: utf-8 -*-
import config
import constants
from core import initialize, observe, update, finalize

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# reload external python modules
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# ==============================================================================
# END: Preamble
# ==============================================================================

# ==============================================================================
# START: Application
# ==============================================================================

# main entry point for the application
# TODO: proper docs
def application():
    
    # pre-conditions
    config.init() # initialize internal config for the app
    time = 0 # define stopwatch for the process
    
     # process for t times
    print('=> START: Running simulation for waterbirds ABM')
    habitats, agents = initialize()
    observe(habitats, agents, time)
    
    for time in range(1, constants.PROCESSING_TIME):
        agents = update(habitats, agents, time) # override agents when being updated
        observe(habitats, agents, time)
    
    print('=> END: Running simulation for waterbirds ABM')
    
    # post-conditions
    finalize()

    
# run application
application()

# ==============================================================================
# END: Application
# ==============================================================================

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=> START: Running simulation for waterbirds ABM
==> 7 habitats have been created successfully!
==> 22 agents have been created successfully!
--- snapshot for time 1
--- updating agents will start processing...
--- snapshot for time 2
--- snapshot for time 3
--- snapshot for time 4
--- snapshot for time 5
--- snapshot for time 6
--- snapshot for time 7
--- snapshot for time 8
--- snapshot for time 9
--- snapshot for time 10
--- snapshot for time 11
--- snapshot for time 12
--- snapshot for time 13
--- snapshot for time 14
--- snapshot for time 15
--- snapshot for time 16
--- snapshot for time 17
--- snapshot for time 18
--- snapshot for time 19
--- snapshot for time 20
--- snapshot for time 21
--- snapshot for time 22
--- snapshot for time 23
--- snapshot for time 24
--- snapshot for time 25
